In [1]:
import pandas as pd
import numpy as np 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import time

In [2]:
train_dataset = pd.read_csv("/Users/vivian/Documents/HKUST Data Driven Modeling/机器学习5054/Final/MNIST/train_resized.csv")
test_dataset = pd.read_csv("/Users/vivian/Documents/HKUST Data Driven Modeling/机器学习5054/Final/MNIST/test_resized.csv")

In [3]:
train_dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel135,pixel136,pixel137,pixel138,pixel139,pixel140,pixel141,pixel142,pixel143,pixel144
0,2,0.0,0.0,0.072606,-0.360184,-6.625228,-3.997078,-5.531005,-1.731706,0.166274,...,0.000000,0.000000,0.000000,0.000075,0.015430,-0.189739,-0.185194,0.016683,0.000026,0.0
1,1,0.0,0.0,0.000000,0.001739,0.313278,-3.799580,-6.506716,0.240206,0.010433,...,0.000000,0.010356,-0.087299,-2.982227,-0.779243,0.061083,0.000609,0.000000,0.000000,0.0
2,1,0.0,0.0,0.000000,0.045693,-0.252522,-6.744340,-0.851903,0.112032,0.000000,...,0.000000,0.000000,0.168599,-1.623264,-4.564190,0.095272,0.009263,0.000000,0.000000,0.0
3,5,0.0,0.0,0.000000,0.091467,-0.728588,-6.229659,-1.798628,0.033384,-0.630858,...,0.011929,-0.137988,-0.420487,-0.096168,0.009753,0.000000,0.000000,0.000000,0.000000,0.0
4,3,0.0,0.0,0.000000,0.000862,0.170443,-1.632121,-9.145603,-5.091849,0.222623,...,-0.155082,-1.367886,-4.408230,-2.914262,0.112931,0.010156,0.000000,0.000000,0.000000,0.0


In [4]:
x_train=train_dataset.drop(columns=["label"])
y_train=train_dataset["label"]
X_train=(x_train.to_numpy()-min(x_train.min()))/(max(x_train.max())-min(x_train.min()))
X_train = X_train.reshape(-1, 12, 12, 1)
Y_train=y_train.to_numpy()

x_test=test_dataset.drop(columns=["label"])
y_test=test_dataset["label"]
X_test=(x_test.to_numpy()-min(x_test.min()))/(max(x_test.max())-min(x_test.min()))
X_test = X_test.reshape(-1, 12, 12, 1)
Y_test=y_test.to_numpy()

In [5]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(30000, 12, 12, 1)
(30000,)
(12000, 12, 12, 1)
(12000,)


In [8]:
# one-hot encode the labels
# 将Y_train转换为PyTorch的Tensor
Y_train_tensor = torch.from_numpy(Y_train)
# torch.nn.functional.one_hot进行one-hot编码
Y_train_onehot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=10)

print(Y_train_onehot.shape)

torch.Size([30000, 10])


In [9]:
import torch
import torch.nn as nn

# 定义CNN模型
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 192, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(192, 192, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool2d(2, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(192 * 4 * 4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = self.pool(x)
        x = self.relu(self.conv5(x))
        x = self.pool2(self.relu(self.conv6(x)))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.softmax(self.fc2(x))
        return x

# 创建CNN模型实例
model = CNN()

# 打印模型结构
print(model)

CNN(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(192, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3072, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)
